#### Import Statements

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

#### Loading the dataset

In [2]:
baseline = pd.read_csv('Dataset/baseline_0.csv')
ratings = pd.read_csv('Dataset/ratings.csv')
remarks = pd.read_csv('Dataset/remarks.csv')
remarks_supp_opp = pd.read_csv('Dataset/remarks_supp_opp.csv')
train = pd.read_csv('Dataset/train.csv')
test = pd.read_csv('Dataset/test.csv')

C:\Users\nithy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Removing duplicate entries

In [3]:
print('Ratings: ')
print('Original shape = ', ratings.shape)
ratings.drop_duplicates(inplace=True)
print('Shape after dropping duplicates = ', ratings.shape)
ratings = ratings[(ratings['emp']>0)]
print('Shape after dropping negative emp ids = ', ratings.shape)

print('\nRemarks: ')
print('Original shape = ', remarks.shape)
remarks.drop_duplicates(inplace=True)
print('Shape after dropping duplicates = ', remarks.shape)
remarks = remarks[(remarks['emp']>0)]
print('Shape after dropping negative emp ids = ', remarks.shape)

print('\nRemarks supported opposed: ')
print('Original shape = ', remarks_supp_opp.shape)
remarks_supp_opp.drop_duplicates(inplace=True)
print('Shape after dropping duplicates = ', remarks_supp_opp.shape)
remarks_supp_opp = remarks_supp_opp[(remarks_supp_opp['emp']>0)]
print('Shape after dropping negative emp ids = ', remarks_supp_opp.shape)

print('\nTrain: ')
print('Original shape = ', train.shape)

train.drop_duplicates(inplace=True)
print('Shape after dropping duplicates = ', train.shape)

print('\nTest: ')
print('Original shape = ', test.shape)
test.drop_duplicates(inplace=True)
print('Shape after dropping duplicates = ', test.shape)

Ratings: 
Original shape =  (221232, 4)
Shape after dropping duplicates =  (221145, 4)
Shape after dropping negative emp ids =  (221145, 4)

Remarks: 
Original shape =  (82756, 5)
Shape after dropping duplicates =  (42019, 5)
Shape after dropping negative emp ids =  (38994, 5)

Remarks supported opposed: 
Original shape =  (336960, 5)
Shape after dropping duplicates =  (336953, 5)
Shape after dropping negative emp ids =  (324471, 5)

Train: 
Original shape =  (3526, 5)
Shape after dropping duplicates =  (3526, 5)

Test: 
Original shape =  (882, 4)
Shape after dropping duplicates =  (882, 4)


#### Creating id_list for connecting train and test with rest of the data frames

In [4]:
train_added_features = train.copy()
test_added_features = test.copy()

train_added_features.set_index('id', inplace=True)
test_added_features.set_index('id', inplace=True)


train_added_features.sort_index(inplace=True)
test_added_features.sort_index(inplace=True)

In [5]:
# To check for duplicate entries with different ids

print(np.unique(train_added_features.duplicated()))
print(np.unique(test_added_features.duplicated()))

train_added_features.drop_duplicates(inplace=True)
test_added_features.drop_duplicates(inplace=True)

print('\n')
print(np.unique(train_added_features.duplicated()))
print(np.unique(test_added_features.duplicated()))

[False  True]
[False]


[False]
[False]


In [6]:
train_added_features['id'] = train_added_features.index.copy()
test_added_features['id'] = test_added_features.index.copy()

In [7]:
id_list_train = pd.DataFrame(train_added_features.drop(['lastratingdate','left'],axis=1))
id_list_test = pd.DataFrame(test_added_features.drop('lastratingdate',axis=1))


# Creating id_list for connecting train and test with rest of the data frames
id_list = pd.DataFrame(np.vstack([train_added_features.drop(['lastratingdate','left'],axis=1),test_added_features.drop('lastratingdate',axis=1)]))
id_list.columns = id_list_train.columns

In [8]:
multiple_entries = []

In [9]:
id_list.groupby(['comp','emp']).apply(lambda x: multiple_entries.append((x.comp.values[0], x.emp.values[0])) if x.id.count()!=1 else np.nan)

comp      emp
azalutpt  1     NaN
          3     NaN
          4     NaN
          6     NaN
          7     NaN
                 ..
yodaczsb  313   NaN
          315   NaN
          319   NaN
zptfoxyq  17    NaN
          19    NaN
Length: 4374, dtype: float64

In [10]:
for me in multiple_entries:
    print(train_added_features[(train_added_features['emp']==me[1])&(train_added_features['comp']==me[0])])

    emp      comp lastratingdate  left  id
id                                        
24   14  bnivzbfi     04-03-2016     1  24
     emp      comp lastratingdate  left   id
id                                          
8    475  bnivzbfi     06-11-2016     1    8
340  475  bnivzbfi     12-01-2017     1  340
     emp      comp lastratingdate  left   id
id                                          
347  487  bnivzbfi     12-12-2016     1  347
     emp      comp lastratingdate  left   id
id                                          
946  135  ewpvmfbc     21-04-2016     1  946
      emp      comp lastratingdate  left    id
id                                            
3026    2  lgqwnfsg     10-03-2017     1  3026
3027    2  lgqwnfsg     11-03-2017     1  3027
      emp      comp lastratingdate  left    id
id                                            
3028   19  lgqwnfsg     21-02-2017     1  3028
3043   19  lgqwnfsg     23-02-2017     1  3043
     emp      comp lastratingdate  left   id


#### Looks like people with reused ids in a company left!

In [11]:
for me in multiple_entries:
    try:
        train_added_features.loc[(train_added_features['emp']==me[1])&(train_added_features['comp']==me[0]), 'Reused_id'] = 1
    except:
        pass
    try:
        test_added_features.loc[(test_added_features['emp']==me[1])&(test_added_features['comp']==me[0]), 'Reused_id'] = 1
    except:
        pass

train_added_features.replace(to_replace=np.nan,value=0,inplace=True)
test_added_features.replace(to_replace=np.nan,value=0,inplace=True)

#### Adding number of ratings

In [12]:
# Adding test ids for ratings
ratings_train = id_list_train.merge(ratings, how='outer')

# Removing rows with nan ids
ratings_train = ratings_train[~ratings_train['id'].isnull()] 
ratings_train.sort_values('id',inplace=True)
ratings_train.head()

,emp,comp,id,Date,rating
0,512,bnivzbfi,0.0,17-01-2017,3
1,512,bnivzbfi,0.0,23-01-2017,2
2,512,bnivzbfi,0.0,21-02-2017,1
3,512,bnivzbfi,0.0,23-02-2017,4
25,3,bnivzbfi,4.0,16-02-2017,3


In [13]:
# Adding test ids for ratings
ratings_test = id_list_test.merge(ratings, how='outer')

# Removing rows with nan ids
ratings_test = ratings_test[~ratings_test['id'].isnull()] 
ratings_test.sort_values('id',inplace=True)
ratings_test.head()

,emp,comp,id,Date,rating
0,2,bnivzbfi,2.0,05-02-2016,2
52,2,bnivzbfi,2.0,11-10-2016,2
51,2,bnivzbfi,2.0,07-10-2016,3
50,2,bnivzbfi,2.0,06-10-2016,1
49,2,bnivzbfi,2.0,29-09-2016,1


In [14]:
rating_max = int(max(ratings_train['rating'].values))

for i in range(1, rating_max+1):
    col = '{}_ratings'.format(i)
    train_added_features[col] = ratings_train[ratings_train['rating']==i].groupby('id')['rating'].count()

train_added_features.replace(to_replace=np.nan,value=0,inplace=True)

In [15]:
train_added_features['total_num_ratings'] = train_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']].sum(axis = 1)
train_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']] = train_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']].divide(train_added_features['total_num_ratings'], axis=0)
train_added_features.head()

,emp,comp,lastratingdate,left,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings
id,,,,,,,,,,,
0,512,bnivzbfi,23-02-2017,0,0,0.0,0.250000,0.250000,0.250000,0.250000,4.0
4,3,bnivzbfi,16-02-2017,0,4,0.0,0.000000,0.136364,0.272727,0.590909,22.0
6,4,bnivzbfi,20-03-2017,0,6,0.0,0.015385,0.025641,0.046154,0.912821,195.0
7,516,bnivzbfi,20-03-2017,0,7,0.0,0.137931,0.172414,0.448276,0.241379,29.0
8,475,bnivzbfi,06-11-2016,1,8,1.0,0.066667,0.200000,0.600000,0.133333,15.0


In [16]:
for i in range(1, rating_max+1):
    col = '{0}_ratings'.format(i)
    test_added_features[col] = ratings_test[ratings_test['rating']==i].groupby('id')['rating'].count()

test_added_features.replace(to_replace=np.nan,value=0,inplace=True)

In [17]:
test_added_features['total_num_ratings'] = test_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']].sum(axis = 1)
test_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']] = test_added_features[['1_ratings', '2_ratings', '3_ratings', '4_ratings']].divide(test_added_features['total_num_ratings'], axis=0)
test_added_features.head()

,emp,comp,lastratingdate,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings
id,,,,,,,,,,
2,2,bnivzbfi,17-03-2017,2,0.0,0.305556,0.263889,0.305556,0.125000,72.0
3,487,bnivzbfi,19-11-2016,3,1.0,0.000000,0.071429,0.714286,0.214286,14.0
14,14,bnivzbfi,04-03-2016,14,1.0,0.000000,0.111111,0.555556,0.333333,9.0
28,18,bnivzbfi,20-03-2017,28,0.0,0.105882,0.223529,0.329412,0.341176,170.0
31,20,bnivzbfi,15-11-2016,31,0.0,0.000000,0.148649,0.283784,0.567568,74.0


#### Adding last rating and fixing last rating date

In [18]:
train_added_features['lastratingdate'] = pd.to_datetime(train_added_features['lastratingdate']).dt.strftime('%Y-%m-%d') 
test_added_features['lastratingdate'] = pd.to_datetime(test_added_features['lastratingdate']).dt.strftime('%Y-%m-%d')

In [19]:
ratings_train["Date"] = pd.to_datetime(ratings_train["Date"]).dt.strftime('%Y-%m-%d')
ratings_train.sort_values(by='Date',ascending=False,inplace=True)

In [20]:
ratings_test["Date"] = pd.to_datetime(ratings_test["Date"]).dt.strftime('%Y-%m-%d')
ratings_test.sort_values(by='Date',ascending=False,inplace=True)

In [21]:
# Making corrections to the last date mentioned in the train dataset
lastdate = ratings_train.groupby('id')['Date'].first()
train_added_features.loc[~(train_added_features['lastratingdate']>= lastdate), 'lastratingdate'] = lastdate[~(train_added_features['lastratingdate']>= lastdate)]

# Adding the last rating given by an employee to the train set
train_added_features['lastrating'] = ratings_train.groupby('id')['rating'].first().values
train_added_features.head()

,emp,comp,lastratingdate,left,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating
id,,,,,,,,,,,,
0,512,bnivzbfi,2017-02-23,0,0,0.0,0.250000,0.250000,0.250000,0.250000,4.0,4
4,3,bnivzbfi,2017-09-02,0,4,0.0,0.000000,0.136364,0.272727,0.590909,22.0,4
6,4,bnivzbfi,2017-12-03,0,6,0.0,0.015385,0.025641,0.046154,0.912821,195.0,4
7,516,bnivzbfi,2017-12-03,0,7,0.0,0.137931,0.172414,0.448276,0.241379,29.0,4
8,475,bnivzbfi,2017-12-01,1,8,1.0,0.066667,0.200000,0.600000,0.133333,15.0,3


In [22]:
# Making corrections to the last date mentioned in the test dataset
lastdate = ratings_test.groupby('id')['Date'].first()
test_added_features.loc[~(test_added_features['lastratingdate']>= lastdate), 'lastratingdate'] = lastdate[~(test_added_features['lastratingdate']>= lastdate)]

# Adding the last rating given by an employee to the test set
test_added_features['lastrating'] = ratings_test.groupby('id')['rating'].first().values
test_added_features.head()

,emp,comp,lastratingdate,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating
id,,,,,,,,,,,
2,2,bnivzbfi,2017-03-17,2,0.0,0.305556,0.263889,0.305556,0.125000,72.0,3
3,487,bnivzbfi,2016-12-12,3,1.0,0.000000,0.071429,0.714286,0.214286,14.0,3
14,14,bnivzbfi,2016-11-02,14,1.0,0.000000,0.111111,0.555556,0.333333,9.0,3
28,18,bnivzbfi,2017-12-03,28,0.0,0.105882,0.223529,0.329412,0.341176,170.0,3
31,20,bnivzbfi,2016-12-07,31,0.0,0.000000,0.148649,0.283784,0.567568,74.0,4


In [23]:
train_added_features['lastratingdate'] = train_added_features['lastratingdate'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d').date())
test_added_features['lastratingdate'] = test_added_features['lastratingdate'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d').date())

#### Adding Delta Days

In [24]:
ratings["Date"] = pd.to_datetime(ratings["Date"]).dt.strftime('%Y-%m-%d')
ratings.sort_values(by='Date',ascending=False,inplace=True)
comp_lastratingdate = ratings.groupby('comp')['Date'].first().apply(lambda d: datetime.strptime(d, '%Y-%m-%d').date())

In [25]:
comps_train = np.unique(train_added_features['comp'].values)
comps_test = np.unique(test_added_features['comp'].values)

for c in comps_train:
    train_added_features.loc[train_added_features['comp']==c, 'Delta_days'] = comp_lastratingdate[c] - train_added_features[train_added_features['comp']==c]['lastratingdate']
for c in comps_test:    
    test_added_features.loc[test_added_features['comp']==c, 'Delta_days'] = comp_lastratingdate[c] - test_added_features[test_added_features['comp']==c]['lastratingdate'] 

In [26]:
train_added_features['Delta_days'] = train_added_features['Delta_days'].apply(lambda x: x.days)
test_added_features['Delta_days'] = test_added_features['Delta_days'].apply(lambda x: x.days)

In [27]:
np.mean(train_added_features['Delta_days'][train_added_features['left']==1].values)

365.2289156626506

In [28]:
np.mean(train_added_features['Delta_days'][train_added_features['left']==0].values)

147.97373806275579

#### Delta days is higher for employess who left the company! Awesome!

## Adding Supported Ratings

In [29]:
# # Adding train ids for remarks_ratings
# remarks_ratings_train = id_list_train.merge(remarks_ratings, how='outer')

# # Removing rows with nan ids
# remarks_ratings_train = remarks_ratings_train[~remarks_ratings_train['id'].isnull()] 
# remarks_ratings_train.sort_values('id',inplace=True)
# remarks_ratings_train.head()

# # Adding test ids for remarks_ratings
# remarks_ratings_test = id_list_test.merge(remarks_ratings, how='outer')

# # Removing rows with nan ids
# remarks_ratings_test = remarks_ratings_test[~remarks_ratings_test['id'].isnull()] 
# remarks_ratings_test.sort_values('id',inplace=True)
# remarks_ratings_test.head()

In [30]:
remarks.rename({'remarkDate':'Date'}, axis=1, inplace=True)
remarks['Date'] = pd.to_datetime(ratings_train["Date"]).dt.strftime('%Y-%m-%d')
remarks_ratings = ratings.merge(remarks, how='outer')
remarks_ratings['Date'].replace(np.nan, '', inplace = True)
remarks_ratings['Date'] = remarks_ratings['Date'].apply(lambda d: datetime.strptime(d, '%Y-%m-%d').date() if d != '' else '')
remarks_ratings['Length'] = remarks_ratings['txt'].str.len()
remarks_ratings.drop(['txt'], axis=1, inplace=True)
remarks_ratings.head()

,emp,comp,Date,rating,remarkId,Length
0,213,siexkzzo,2017-12-03,3.0,NaN,NaN
1,594,phcvroct,2017-12-03,3.0,NaN,NaN
2,959,phcvroct,2017-12-03,3.0,NaN,NaN
3,949,phcvroct,2017-12-03,3.0,NaN,NaN
4,863,phcvroct,2017-12-03,3.0,NaN,NaN


In [31]:
# Choosing reamrks with ratings
remarks_ratings_non_null = remarks_ratings.loc[remarks_ratings.index[(~remarks_ratings.remarkId.isnull())&(~remarks_ratings.rating.isnull())]]
remarks_ratings_non_null.sort_values(by='remarkId',inplace=True)
remarks_ratings_non_null.set_index('remarkId', inplace=True)

# Computing avg rating for remarkids with multiple ratings
ratings_avg = remarks_ratings_non_null.groupby('remarkId')['rating'].mean().round()
remarks_ratings_non_null = remarks_ratings_non_null.loc[~remarks_ratings_non_null.index.duplicated(keep= "first")]
remarks_ratings_non_null['rating'] = ratings_avg

In [32]:
remarks_ratings_non_null.head()

,emp,comp,Date,rating,Length
remarkId,,,,,
aabtzbxo,768,phcvroct,2017-02-13,3.0,21.0
aadmjbvo,163,bnivzbfi,2016-05-26,4.0,70.0
aafistnf,107,ocsicwng,2016-10-18,3.0,59.0
aakalium,206,bnivzbfi,2016-07-04,3.0,49.0
aallerpm,82,rcwkfavv,2017-03-01,3.0,142.0


In [33]:
# Adding train ids for remarks_supp_opp
supp_train = id_list_train.merge(remarks_supp_opp, how='outer')

# Removing rows with nan ids and nan remark ids
supp_train = supp_train[~supp_train['id'].isnull()] 
supp_train = supp_train[~supp_train['remarkId'].isnull()] 

supp_train.sort_values('id',inplace=True)
supp_train.set_index('id', drop=True, inplace=True)
supp_train['rating']= supp_train.remarkId.apply(lambda r: remarks_ratings_non_null.loc[r,'rating'] if r in remarks_ratings_non_null.index else np.nan)
supp_train.head()

,emp,comp,support,oppose,remarkId,rating
id,,,,,,
0.0,512,bnivzbfi,True,False,iyuuhtdh,NaN
0.0,512,bnivzbfi,True,False,wmtumnwb,NaN
0.0,512,bnivzbfi,True,False,wqxoqxky,NaN
0.0,512,bnivzbfi,True,False,ungzilbe,NaN
0.0,512,bnivzbfi,True,False,vitxsibj,NaN


In [34]:
# Adding test ids for remarks_supp_opp
supp_test = id_list_test.merge(remarks_supp_opp, how='outer')

# Removing rows with nan ids and nan remark ids
supp_test = supp_test[~supp_test['id'].isnull()] 
supp_test = supp_test[~supp_test['remarkId'].isnull()] 

supp_test.sort_values('id',inplace=True)
supp_test.set_index('id', drop=True, inplace=True)
supp_test['rating']= supp_test.remarkId.apply(lambda r: remarks_ratings_non_null.loc[r,'rating'] if r in remarks_ratings_non_null.index else np.nan)
supp_test.head()

,emp,comp,support,oppose,remarkId,rating
id,,,,,,
2.0,2,bnivzbfi,True,False,iyuuhtdh,NaN
2.0,2,bnivzbfi,True,False,tsnpxtoo,NaN
2.0,2,bnivzbfi,True,False,tocxzomp,4.0
2.0,2,bnivzbfi,False,True,kcnjqgts,NaN
2.0,2,bnivzbfi,True,False,efimyjbf,NaN


In [35]:
train_added_features['Supported'] = supp_train[supp_train['support']==True].groupby('id')['support'].count()
train_added_features['Opposed'] = supp_train[supp_train['oppose']==True].groupby('id')['oppose'].count()
    
test_added_features['Supported'] = supp_test[supp_test['support']==True].groupby('id')['support'].count()
test_added_features['Opposed'] = supp_test[supp_test['oppose']==True].groupby('id')['oppose'].count()

In [36]:
for i in range(1, rating_max+1):
    train_added_features['{}_reviews_supported'.format(i)] = supp_train[(supp_train['rating']==i)&(supp_train['support']==True)].groupby('id')['rating'].count()
    train_added_features['{}_reviews_opposed'.format(i)] = supp_train[(supp_train['rating']==i)&(supp_train['oppose']==True)].groupby('id')['rating'].count()
    

    test_added_features['{}_reviews_supported'.format(i)] = supp_test[(supp_test['rating']==i)&(supp_test['support']==True)].groupby('id')['rating'].count()
    test_added_features['{}_reviews_opposed'.format(i)] = supp_test[(supp_test['rating']==i)&(supp_test['oppose']==True)].groupby('id')['rating'].count()

train_added_features.replace(to_replace=np.nan,value=0,inplace=True)
test_added_features.replace(to_replace=np.nan,value=0,inplace=True)

In [37]:
train_added_features['total_opp'] = train_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']].sum(axis = 1)
train_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']] = train_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']].divide(train_added_features['total_opp'], axis=0)

train_added_features['total_supp'] = train_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']].sum(axis = 1)
train_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']] = train_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']].divide(train_added_features['total_supp'], axis=0)
train_added_features.head()

,emp,comp,lastratingdate,left,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,...,1_reviews_supported,1_reviews_opposed,2_reviews_supported,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp
id,,,,,,,,,,,,,,,,,,,,,
0,512,bnivzbfi,2017-02-23,0,0,0.0,0.250000,0.250000,0.250000,0.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,3,bnivzbfi,2017-09-02,0,4,0.0,0.000000,0.136364,0.272727,0.590909,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0
6,4,bnivzbfi,2017-12-03,0,6,0.0,0.015385,0.025641,0.046154,0.912821,...,0.151515,0.166667,0.090909,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0
7,516,bnivzbfi,2017-12-03,0,7,0.0,0.137931,0.172414,0.448276,0.241379,...,0.166667,0.333333,0.222222,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0
8,475,bnivzbfi,2017-12-01,1,8,1.0,0.066667,0.200000,0.600000,0.133333,...,0.416667,NaN,0.000000,NaN,0.083333,NaN,0.500000,NaN,0.0,12.0


In [38]:
test_added_features['total_opp'] = test_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']].sum(axis = 1)
test_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']] = test_added_features[['1_reviews_opposed', '2_reviews_opposed', '3_reviews_opposed', '4_reviews_opposed']].divide(test_added_features['total_opp'], axis=0)


test_added_features['total_supp'] = test_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']].sum(axis = 1)
test_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']] = test_added_features[['1_reviews_supported', '2_reviews_supported', '3_reviews_supported', '4_reviews_supported']].divide(test_added_features['total_supp'], axis=0)
test_added_features.head()

,emp,comp,lastratingdate,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,...,1_reviews_supported,1_reviews_opposed,2_reviews_supported,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp
id,,,,,,,,,,,,,,,,,,,,,
2,2,bnivzbfi,2017-03-17,2,0.0,0.305556,0.263889,0.305556,0.125000,72.0,...,0.210526,0.000000,0.105263,0.000000,0.000000,1.00,0.684211,0.000,3.0,19.0
3,487,bnivzbfi,2016-12-12,3,1.0,0.000000,0.071429,0.714286,0.214286,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
14,14,bnivzbfi,2016-11-02,14,1.0,0.000000,0.111111,0.555556,0.333333,9.0,...,0.000000,NaN,0.000000,NaN,0.000000,NaN,1.000000,NaN,0.0,2.0
28,18,bnivzbfi,2017-12-03,28,0.0,0.105882,0.223529,0.329412,0.341176,170.0,...,0.137037,0.200000,0.114815,0.075000,0.162963,0.15,0.585185,0.575,40.0,270.0
31,20,bnivzbfi,2016-12-07,31,0.0,0.000000,0.148649,0.283784,0.567568,74.0,...,0.097087,0.133333,0.106796,0.266667,0.194175,0.20,0.601942,0.400,15.0,103.0


In [39]:
# Filling the missing values of supported and opposed
null_ids_train = train_added_features[train_added_features['1_reviews_supported'].isnull()].id
null_ids_test = test_added_features[test_added_features['1_reviews_supported'].isnull()].id

In [40]:
for i in range(1,5):
    train_added_features.loc[null_ids_train,'{}_reviews_supported'.format(i)] = train_added_features.loc[null_ids_train,'{}_ratings'.format(i)]
    test_added_features.loc[null_ids_test,'{}_reviews_supported'.format(i)] = test_added_features.loc[null_ids_test,'{}_ratings'.format(i)]

In [41]:
train_added_features.head()

,emp,comp,lastratingdate,left,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,...,1_reviews_supported,1_reviews_opposed,2_reviews_supported,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp
id,,,,,,,,,,,,,,,,,,,,,
0,512,bnivzbfi,2017-02-23,0,0,0.0,0.250000,0.250000,0.250000,0.250000,...,0.250000,NaN,0.250000,NaN,0.250000,NaN,0.250000,NaN,0.0,0.0
4,3,bnivzbfi,2017-09-02,0,4,0.0,0.000000,0.136364,0.272727,0.590909,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0
6,4,bnivzbfi,2017-12-03,0,6,0.0,0.015385,0.025641,0.046154,0.912821,...,0.151515,0.166667,0.090909,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0
7,516,bnivzbfi,2017-12-03,0,7,0.0,0.137931,0.172414,0.448276,0.241379,...,0.166667,0.333333,0.222222,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0
8,475,bnivzbfi,2017-12-01,1,8,1.0,0.066667,0.200000,0.600000,0.133333,...,0.416667,NaN,0.000000,NaN,0.083333,NaN,0.500000,NaN,0.0,12.0


In [42]:
null_ids_train = train_added_features[train_added_features['1_reviews_opposed'].isnull()].id
null_ids_test = test_added_features[test_added_features['1_reviews_opposed'].isnull()].id

In [43]:
for i in range(1,5):
    j = 5-i
    train_added_features.loc[null_ids_train,'{}_reviews_opposed'.format(i)] = train_added_features.loc[null_ids_train,'{}_reviews_supported'.format(j)]
    test_added_features.loc[null_ids_test,'{}_reviews_opposed'.format(i)] = test_added_features.loc[null_ids_test,'{}_reviews_supported'.format(j)]

In [44]:
train_added_features.head()

,emp,comp,lastratingdate,left,id,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,...,1_reviews_supported,1_reviews_opposed,2_reviews_supported,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp
id,,,,,,,,,,,,,,,,,,,,,
0,512,bnivzbfi,2017-02-23,0,0,0.0,0.250000,0.250000,0.250000,0.250000,...,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.0,0.0
4,3,bnivzbfi,2017-09-02,0,4,0.0,0.000000,0.136364,0.272727,0.590909,...,0.000000,0.000000,0.333333,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0
6,4,bnivzbfi,2017-12-03,0,6,0.0,0.015385,0.025641,0.046154,0.912821,...,0.151515,0.166667,0.090909,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0
7,516,bnivzbfi,2017-12-03,0,7,0.0,0.137931,0.172414,0.448276,0.241379,...,0.166667,0.333333,0.222222,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0
8,475,bnivzbfi,2017-12-01,1,8,1.0,0.066667,0.200000,0.600000,0.133333,...,0.416667,0.500000,0.000000,0.083333,0.083333,0.000000,0.500000,0.416667,0.0,12.0


1. enocde emp,comp
2. remove lastraatingdate
3. use ratings fraction for reviews supported, 1- for ratings opposed & normalise /group by company and take avg for reviews supported & opposed (done)
4. reliability 0.5 (done)


In [45]:
# Cleaning the remarks file
remarks_ids = id_list.merge(remarks, how = 'outer')
remarks_ids.sort_values('id',inplace = True)
remarks_ids = remarks_ids[~remarks_ids['remarkId'].isnull()]
remarks_ids = remarks_ids[~(remarks_ids['id'].isnull())] 
remarks_ids.drop_duplicates(inplace=True)
remarks_ids.index = remarks_ids.remarkId.values

####################################################################################################################
remarks_ids = remarks_ids.loc[~remarks_ids.index.duplicated(keep = 'first')] 

remarks_ids.head()
remarks_ids.shape

(38979, 6)

In [46]:
# multiple_entries

In [47]:
# remarks_ids.loc['myslcihg', 'id']

In [48]:
# remarks_ids[remarks_ids.index.duplicated()]

#### Adding Reliability

In [49]:
# Adding corresponding id beside every remarkid to identify which person it belongs to
remarks_supp_opp_labelled = remarks_supp_opp[~remarks_supp_opp['remarkId'].isnull()]
remarks_supp_opp_labelled['id'] = remarks_supp_opp_labelled.remarkId.apply(lambda r: remarks_ids.loc[r,'id'] if r in remarks_ids.index else '')

remarks_supp = remarks_supp_opp_labelled[remarks_supp_opp_labelled['support']==True]
remarks_opp = remarks_supp_opp_labelled[remarks_supp_opp_labelled['oppose']==True]

In [50]:
# Adding the Reliability and Non reliability 
counts_supp = remarks_supp['id'].value_counts()
counts_opp = remarks_opp['id'].value_counts()
train_added_features['Reliability'] = train_added_features.id.apply(lambda r: counts_supp[r] if r in counts_supp.index else 0)
train_added_features['NonReliability'] = train_added_features.id.apply(lambda r: counts_opp[r] if r in counts_opp.index else 0)

In [51]:
# Normalizing reliability
train_added_features['Reliability_factor'] = train_added_features['Reliability']/(train_added_features['Reliability']+train_added_features['NonReliability'])
train_added_features['Reliability_factor'].replace(to_replace=np.nan, value=0.5,inplace=True)

In [52]:
test_added_features['Reliability'] = test_added_features.id.apply(lambda r: counts_supp[r] if r in counts_supp.index else 0)
test_added_features['NonReliability'] = test_added_features.id.apply(lambda r: counts_opp[r] if r in counts_opp.index else 0)

In [53]:
# Normalizing reliability
test_added_features['Reliability_factor'] = test_added_features['Reliability']/(test_added_features['Reliability']+test_added_features['NonReliability'])
test_added_features['Reliability_factor'].replace(to_replace=np.nan, value=0.5,inplace=True)

In [54]:
train_added_features.drop(['id', 'lastratingdate', 'emp'], axis=1, inplace=True)
test_added_features.drop(['id', 'lastratingdate', 'emp'], axis=1, inplace=True)

In [55]:
train_added_features.head()

,comp,left,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating,Delta_days,...,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp,Reliability,NonReliability,Reliability_factor
id,,,,,,,,,,,,,,,,,,,,,
0,bnivzbfi,0,0.0,0.250000,0.250000,0.250000,0.250000,4.0,4,283,...,0.250000,0.250000,0.250000,0.250000,0.250000,0.0,0.0,15,2,0.882353
4,bnivzbfi,0,0.0,0.000000,0.136364,0.272727,0.590909,22.0,4,92,...,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0,30,8,0.789474
6,bnivzbfi,0,0.0,0.015385,0.025641,0.046154,0.912821,195.0,4,0,...,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0,80,10,0.888889
7,bnivzbfi,0,0.0,0.137931,0.172414,0.448276,0.241379,29.0,4,0,...,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0,69,32,0.683168
8,bnivzbfi,1,1.0,0.066667,0.200000,0.600000,0.133333,15.0,3,2,...,0.083333,0.083333,0.000000,0.500000,0.416667,0.0,12.0,65,72,0.474453


In [56]:
test_added_features.head()

,comp,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating,Delta_days,Supported,...,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp,Reliability,NonReliability,Reliability_factor
id,,,,,,,,,,,,,,,,,,,,,
2,bnivzbfi,0.0,0.305556,0.263889,0.305556,0.125000,72.0,3,261,60.0,...,0.000000,0.000000,1.000000,0.684211,0.000,3.0,19.0,76,67,0.531469
3,bnivzbfi,1.0,0.000000,0.071429,0.714286,0.214286,14.0,3,356,1.0,...,0.714286,0.714286,0.071429,0.214286,0.000,0.0,0.0,6,0,1.000000
14,bnivzbfi,1.0,0.000000,0.111111,0.555556,0.333333,9.0,3,396,15.0,...,0.000000,0.000000,0.000000,1.000000,0.000,0.0,2.0,10,1,0.909091
28,bnivzbfi,0.0,0.105882,0.223529,0.329412,0.341176,170.0,3,0,745.0,...,0.075000,0.162963,0.150000,0.585185,0.575,40.0,270.0,671,260,0.720730
31,bnivzbfi,0.0,0.000000,0.148649,0.283784,0.567568,74.0,4,361,254.0,...,0.266667,0.194175,0.200000,0.601942,0.400,15.0,103.0,410,86,0.826613


In [57]:
def encoding(train_data, cat_var, test_data):
    
    counts = train_data.groupby(cat_var)['left'].count()
    means = train_data.groupby(cat_var)['left'].mean()
    
#     m = 0
#     smooth_mean = (counts * means + m * train_data[cat_var].mean()) / (counts + m)
#     enc = dict(smooth_mean)

    enc = dict(means)
    train_data[cat_var] = train_data[cat_var].map(enc)
    test_data[cat_var] = test_data[cat_var].map(enc)
   
#     try:
#         if test_data==None:
#             return 1
#     except:
#         pass
#     else:
#         test_data[cat_var] = test_data[cat_var].map(enc)
#         return 1  

In [58]:
encoding(train_added_features, 'comp', test_added_features)

In [59]:
train_added_features.head()

,comp,left,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating,Delta_days,...,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp,Reliability,NonReliability,Reliability_factor
id,,,,,,,,,,,,,,,,,,,,,
0,0.066176,0,0.0,0.250000,0.250000,0.250000,0.250000,4.0,4,283,...,0.250000,0.250000,0.250000,0.250000,0.250000,0.0,0.0,15,2,0.882353
4,0.066176,0,0.0,0.000000,0.136364,0.272727,0.590909,22.0,4,92,...,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0,30,8,0.789474
6,0.066176,0,0.0,0.015385,0.025641,0.046154,0.912821,195.0,4,0,...,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0,80,10,0.888889
7,0.066176,0,0.0,0.137931,0.172414,0.448276,0.241379,29.0,4,0,...,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0,69,32,0.683168
8,0.066176,1,1.0,0.066667,0.200000,0.600000,0.133333,15.0,3,2,...,0.083333,0.083333,0.000000,0.500000,0.416667,0.0,12.0,65,72,0.474453


In [60]:
test_added_features.head()

,comp,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating,Delta_days,Supported,...,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp,Reliability,NonReliability,Reliability_factor
id,,,,,,,,,,,,,,,,,,,,,
2,0.066176,0.0,0.305556,0.263889,0.305556,0.125000,72.0,3,261,60.0,...,0.000000,0.000000,1.000000,0.684211,0.000,3.0,19.0,76,67,0.531469
3,0.066176,1.0,0.000000,0.071429,0.714286,0.214286,14.0,3,356,1.0,...,0.714286,0.714286,0.071429,0.214286,0.000,0.0,0.0,6,0,1.000000
14,0.066176,1.0,0.000000,0.111111,0.555556,0.333333,9.0,3,396,15.0,...,0.000000,0.000000,0.000000,1.000000,0.000,0.0,2.0,10,1,0.909091
28,0.066176,0.0,0.105882,0.223529,0.329412,0.341176,170.0,3,0,745.0,...,0.075000,0.162963,0.150000,0.585185,0.575,40.0,270.0,671,260,0.720730
31,0.066176,0.0,0.000000,0.148649,0.283784,0.567568,74.0,4,361,254.0,...,0.266667,0.194175,0.200000,0.601942,0.400,15.0,103.0,410,86,0.826613


In [62]:
train_added_features.to_csv('Train_added_features_final.csv')
test_added_features.to_csv('Test_added_features_final.csv')